In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV
pd.options.mode.chained_assignment = None

In [56]:
class DataLoader():

    # data preprocessing and loading
    def __init__(self, data_path):

        # load data and format
        sku_info_df = pd.read_csv(data_path + '/jdata_sku_basic_info.csv')
        user_action_df = pd.read_csv(data_path + '/jdata_user_action.csv')
        user_info_df = pd.read_csv(data_path + '/jdata_user_basic_info.csv')
        user_score_df = pd.read_csv(data_path + '/jdata_user_comment_score.csv')
        user_order_df = pd.read_csv(data_path + '/jdata_user_order.csv')
        user_action_df.a_date = pd.to_datetime(user_action_df.a_date)
        user_score_df.comment_create_tm = pd.to_datetime(user_score_df.comment_create_tm)
        user_order_df.o_date = pd.to_datetime(user_order_df.o_date)

        # user table
        user_df = user_info_df

        # sku table
        sku_df = sku_info_df.drop(['para_1', 'para_2', 'para_3'], axis=1)
        sku_cate_df = sku_df[['sku_id', 'cate']].drop_duplicates()

        # action table
        action_df = user_action_df.merge(sku_cate_df, on='sku_id', how='left')
        action_df['a_month'] = action_df.a_date.dt.month
        action_df['a_day'] = action_df.a_date.dt.day

        # order table
        order_df = user_order_df.merge(sku_cate_df, on='sku_id', how='left')
        score_df = user_score_df.drop('comment_create_tm', axis=1)
        order_df = order_df.merge(score_df, on=['o_id', 'user_id'], how='left').fillna(-1)
        order_df.score_level = order_df.score_level.astype(int)
        order_df['o_month'] = order_df.o_date.dt.month
        order_df['o_day'] = order_df.o_date.dt.day
        sku_price_df = sku_df[['sku_id', 'price']].drop_duplicates()
        order_df = order_df.merge(sku_price_df, on='sku_id', how='left')
        order_df['o_total_price'] = order_df.price.values * order_df.o_sku_num.values
        order_df = order_df.drop('price', axis=1)

        self.user_df = user_df
        self.sku_df = sku_df
        self.action_df = action_df
        self.order_df = order_df

In [58]:
# load and transform data table
data = DataLoader('data')

In [59]:
data.user_df.head()

,user_id,age,sex,user_lv_cd
0,1,3,2,3
1,2,3,2,3
2,3,3,0,5
3,4,3,2,3
4,5,4,2,2


In [60]:
data.sku_df.head()

,sku_id,price,cate
0,1,364.2,71
1,2,36.7,46
2,3,953.3,71
3,4,110.6,83
4,5,241.5,71


In [61]:
data.action_df.head()

,user_id,sku_id,a_date,a_num,a_type,cate,a_month,a_day
0,1,80036,2017-04-14,4,1,101,4,14
1,1,96959,2017-01-12,2,1,101,1,12
2,1,8017,2017-03-09,1,1,101,3,9
3,1,80036,2017-04-30,6,1,101,4,30
4,1,16607,2017-01-12,3,1,101,1,12


In [62]:
data.order_df.head()

,user_id,sku_id,o_id,o_date,o_area,o_sku_num,cate,score_level,o_month,o_day,o_total_price
0,1,80036,480066,2017-03-09,14,1,101,-1,3,9,511.8
1,1,16607,106933,2017-01-12,14,1,101,-1,1,12,557.0
2,1,80036,542604,2017-04-14,14,1,101,-1,4,14,511.8
3,1,80036,196335,2017-04-30,14,1,101,-1,4,30,511.8
4,1,80036,42502,2017-03-22,14,2,101,-1,3,22,1023.6


In [44]:
# various date ranges for train/test
featured_month_periods = [1, 3, 6]

test_label_begin_date = datetime(2017, 5, 1)
test_label_end_date = test_label_begin_date + relativedelta(months=1, days=-1)
test_feature_begin_dates = [test_label_begin_date - relativedelta(months=i) for i in featured_month_periods]
test_feature_end_date = test_label_begin_date - relativedelta(days=1)
train_label_begin_date = test_label_begin_date - relativedelta(months=1)
train_label_end_date = train_label_begin_date + relativedelta(months=1, days=-1)
train_feature_begin_dates = [train_label_begin_date - relativedelta(months=i) for i in featured_month_periods]
train_feature_end_date = train_label_begin_date - relativedelta(days=1)

In [45]:
# convenient function for feature merging
def merge_df(df1, df2):
    return df2 if df1 is None else df1.merge(df2, on='user_id', how='left')

In [46]:
# get action features
def get_action_feature(data, begin_date, end_date, month_period):

    action_feature_df = data.user_df[['user_id']]

    # filter by date range
    df = data.action_df[(data.action_df.a_date >= begin_date) & (data.action_df.a_date <= end_date)]

    # logical conditions
    cate_30_cond = (df.cate == 30)
    cate_101_cond = (df.cate == 101)
    cate_30_101_cond = (df.cate == 30) | (df.cate == 101)
    cate_others_cond = (df.cate != 30) & (df.cate != 101)

    # feature name prefix
    name = str(month_period) + 'mo_action_'
    name_30 = name + '30_'
    name_101 = name + '101_'
    name_30_101 = name + '30_101_'
    name_others = name + 'others_'
    name_all = name + 'all_'

    ########## FEATURE START ##########

    # action sum: 30 101
    temp_df = df[cate_30_101_cond]. \
                groupby('user_id')['a_num']. \
                sum(). \
                reset_index(). \
                rename(columns={'a_num': name_30_101 + 'sum'})
    action_feature_df = merge_df(action_feature_df, temp_df)

    # action count: 30 101
    temp_df = df[cate_30_101_cond]. \
                groupby('user_id')['a_num']. \
                count(). \
                reset_index(). \
                rename(columns={'a_num': name_30_101 + 'count'})
    action_feature_df = merge_df(action_feature_df, temp_df)

    # action date count: 30 101
    temp_df = df[cate_30_101_cond]. \
                groupby('user_id')['a_date']. \
                count(). \
                reset_index(). \
                rename(columns={'a_date': name_30_101 + 'date_count'})
    action_feature_df = merge_df(action_feature_df, temp_df)

    # average day gap between actions: 30 101
    temp_df = df[cate_30_101_cond]. \
                sort_values('a_date'). \
                groupby('user_id')['a_date']. \
                agg(lambda x: x.diff().mean(skipna=True).days). \
                reset_index(). \
                rename(columns={'a_date': name_30_101 + 'date_gap_mean'})
    action_feature_df = merge_df(action_feature_df, temp_df)

    # most frequent action day: 30 101
    temp_df = df[cate_30_101_cond]. \
                groupby('user_id')['a_day']. \
                agg(lambda x: x.value_counts().index[0]). \
                reset_index(). \
                rename(columns={'a_day': name_30_101 + 'day_freq'})
    action_feature_df = merge_df(action_feature_df, temp_df)

    ########## FEATURE END ##########

    return action_feature_df

In [64]:
# get order features
def get_order_feature(data, begin_date, end_date, month_period):

    order_feature_df = data.user_df[['user_id']]

    # filter by date range
    df = data.order_df[(data.order_df.o_date >= begin_date) & (data.order_df.o_date <= end_date)]

    # logical conditions
    cate_30_cond = (df.cate == 30)
    cate_101_cond = (df.cate == 101)
    cate_30_101_cond = (df.cate == 30) | (df.cate == 101)
    cate_others_cond = (df.cate != 30) & (df.cate != 101)

    # feature name prefix
    name = str(month_period) + 'mo_order_'
    name_30 = name + '30_'
    name_101 = name + '101_'
    name_30_101 = name + '30_101_'
    name_others = name + 'others_'
    name_all = name + 'all_'

    ########## FEATURE START ##########

    # order num: 30 101
    temp_df = df[cate_30_101_cond]. \
                groupby('user_id')['o_id']. \
                nunique(). \
                reset_index(). \
                rename(columns={'o_id': name_30_101 + 'num'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # order num: 30
    temp_df = df[cate_30_cond]. \
                groupby('user_id')['o_id']. \
                nunique(). \
                reset_index(). \
                rename(columns={'o_id': name_30 + 'num'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # order num: 101
    temp_df = df[cate_30_cond]. \
                groupby('user_id')['o_id']. \
                nunique(). \
                reset_index(). \
                rename(columns={'o_id': name_101 + 'num'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # order num: others
    temp_df = df[cate_others_cond]. \
                groupby('user_id')['o_id']. \
                nunique(). \
                reset_index(). \
                rename(columns={'o_id': name_others + 'num'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # order num: all
    temp_df = df. \
                groupby('user_id')['o_id']. \
                nunique(). \
                reset_index(). \
                rename(columns={'o_id': name_all + 'num'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # order item count: 30 101
    temp_df = df[cate_30_101_cond]. \
                groupby('user_id')['sku_id']. \
                count(). \
                reset_index(). \
                rename(columns={'sku_id': name_30_101 + 'item_count'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # order item count: 30
    temp_df = df[cate_30_cond]. \
                groupby('user_id')['sku_id']. \
                count(). \
                reset_index(). \
                rename(columns={'sku_id': name_30 + 'item_count'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # order item count: 101
    temp_df = df[cate_101_cond]. \
                groupby('user_id')['sku_id']. \
                count(). \
                reset_index(). \
                rename(columns={'sku_id': name_101 + 'item_count'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # order item count: others
    temp_df = df[cate_others_cond]. \
                groupby('user_id')['sku_id']. \
                count(). \
                reset_index(). \
                rename(columns={'sku_id': name_others + 'item_count'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # order item num: all
    temp_df = df. \
                groupby('user_id')['sku_id']. \
                nunique(). \
                reset_index(). \
                rename(columns={'sku_id': name_all + 'item_num'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # order date num: 30 101
    temp_df = df[cate_30_101_cond]. \
                groupby('user_id')['o_date']. \
                nunique(). \
                reset_index(). \
                rename(columns={'o_date': name_30_101 + 'date_num'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # order item sum: 30 101
    temp_df = df[cate_30_101_cond]. \
                groupby('user_id')['sku_id']. \
                sum(). \
                reset_index(). \
                rename(columns={'sku_id': name_30_101 + 'item_sum'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # first day of order: 30 101
    temp_df = df[cate_30_101_cond]. \
                groupby('user_id')['o_day']. \
                min(). \
                reset_index(). \
                rename(columns={'o_day': name_30_101 + 'day_first'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # first day of order: 30
    temp_df = df[cate_30_cond]. \
                groupby('user_id')['o_day']. \
                min(). \
                reset_index(). \
                rename(columns={'o_day': name_30 + 'day_first'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # first day of order: 101
    temp_df = df[cate_101_cond]. \
                groupby('user_id')['o_day']. \
                min(). \
                reset_index(). \
                rename(columns={'o_day': name_101 + 'day_first'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # last day of order: 30 101
    temp_df = df[cate_30_101_cond]. \
                groupby('user_id')['o_day']. \
                max(). \
                reset_index(). \
                rename(columns={'o_day': name_30_101 + 'day_last'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # average day of order: 30 101
    temp_df = df[cate_30_101_cond]. \
                groupby('user_id')['o_day']. \
                mean(). \
                reset_index(). \
                rename(columns={'o_day': name_30_101 + 'day_mean'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # order month num: 30 101
    if month_period > 1:
        temp_df = df[cate_30_101_cond]. \
                    groupby('user_id')['o_month']. \
                    nunique(). \
                    reset_index(). \
                    rename(columns={'o_month': name_30_101 + 'month_num'})
        order_feature_df = merge_df(order_feature_df, temp_df)

    # order date gap variance: 30 101
    valid_date_gap_cond = df[cate_30_101_cond]. \
                            groupby('user_id')['o_date']. \
                            nunique(). \
                            reset_index() \
                            ['o_date'] > 2
    temp_df = df[cate_30_101_cond]. \
                sort_values('o_date'). \
                groupby('user_id')['o_date']. \
                agg(lambda x: x.diff().dt.days.std(skipna=True)). \
                reset_index(). \
                rename(columns={'o_date': name_30_101 + 'date_gap_var'}) \
                [valid_date_gap_cond]
    order_feature_df = merge_df(order_feature_df, temp_df)

    # most frequent order area: 30 101
    temp_df = df[cate_30_101_cond]. \
                groupby('user_id')['o_area']. \
                agg(lambda x: x.value_counts().index[0]). \
                reset_index(). \
                rename(columns={'o_area': name_30_101 + 'area_freq'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # total order price: 30 101
    temp_df = df[cate_30_101_cond]. \
                groupby('user_id')['o_total_price']. \
                sum(). \
                reset_index(). \
                rename(columns={'o_total_price': name_30_101 + 'consume_total'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # average order date gap: 30 101
    temp_df = df[cate_30_101_cond]. \
                sort_values('o_date'). \
                groupby('user_id')['o_date']. \
                agg(lambda x: x.diff().mean(skipna=True).days). \
                reset_index(). \
                rename(columns={'o_date': name_30_101 + 'date_gap_mean'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # average order date gap: all
    temp_df = df. \
                sort_values('o_date'). \
                groupby('user_id')['o_date']. \
                agg(lambda x: x.diff().mean(skipna=True).days). \
                reset_index(). \
                rename(columns={'o_date': name_all + 'date_gap_mean'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # last month of order: 30 101
    if month_period > 1:
        temp_df = df[cate_30_101_cond]. \
                    groupby('user_id')['o_month']. \
                    max(). \
                    reset_index(). \
                    rename(columns={'o_month': name_30_101 + 'month_last'})
        order_feature_df = merge_df(order_feature_df, temp_df)

    # most frequent order day: 30 101
    temp_df = df[cate_30_101_cond]. \
                groupby('user_id')['o_day']. \
                agg(lambda x: x.value_counts().index[0]). \
                reset_index(). \
                rename(columns={'o_day': name_30_101 + 'day_freq'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # order score count: 30 101
    temp_df = df[(df.score_level > 0) & cate_30_101_cond]. \
                groupby('user_id')['score_level']. \
                count(). \
                reset_index(). \
                rename(columns={'score_level': 'score_count'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    # good order score count: 30 101
    temp_df = df[(df.score_level == 1) & cate_30_101_cond]. \
                groupby('user_id')['score_level']. \
                count(). \
                reset_index(). \
                rename(columns={'score_level': 'good_score_count'})
    order_feature_df = merge_df(order_feature_df, temp_df)

    ########## FEATURE END ##########

    return order_feature_df

In [79]:
# get action-order-combining features
def get_action_order_feature(data, begin_date, end_date, month_period):

    action_order_feature_df = data.user_df[['user_id']]

    # action-order-combining dataframe
    action_order_df = data.action_df[['user_id', 'sku_id', 'cate', 'a_date']]. \
                        merge(data.order_df[['user_id', 'sku_id', 'cate', 'o_date']], on=['user_id', 'sku_id', 'cate'], how='left')
    action_order_df = action_order_df[action_order_df.o_date >= action_order_df.a_date]
    action_order_df['date_gap'] = (action_order_df.o_date - action_order_df.a_date).apply(lambda x: x.days)

    # filter by date range
    df = action_order_df[(action_order_df.o_date >= begin_date) & (action_order_df.o_date <= end_date) & \
            (action_order_df.a_date >= begin_date) & (action_order_df.a_date <= end_date)]

    # logical conditions
    cate_30_cond = (df.cate == 30)
    cate_101_cond = (df.cate == 101)
    cate_30_101_cond = (df.cate == 30) | (df.cate == 101)
    cate_others_cond = (df.cate != 30) & (df.cate != 101)

    # feature name prefix
    name = str(month_period) + 'mo_action_order_'
    name_30 = name + '30_'
    name_101 = name + '101_'
    name_30_101 = name + '30_101_'
    name_others = name + 'others_'
    name_all = name + 'all_'

    ########## FEATURE START ##########

    # average action to order gap: all
    temp_df = df. \
                groupby('user_id')['date_gap']. \
                mean(). \
                reset_index()
    action_order_feature_df = merge_df(action_order_feature_df, temp_df)

    ########## FEATURE END ##########

    return action_order_feature_df

In [69]:
# get overall features
def get_feature(data, begin_dates, end_date, featured_month_periods):

    # initialize as user features
    feature_df = data.user_df.copy()

    # get feature from different periods and merge together
    for begin_date, month_period in zip(begin_dates, featured_month_periods):

        # get features from action table and order table in particular period
        action_feature_df = get_action_feature(data, begin_date, end_date, month_period)
        order_feature_df = get_order_feature(data, begin_date, end_date, month_period)
        action_order_feature_df = get_action_order_feature(data, begin_date, end_date, month_period)

        # merge action and order features
        feature_df = merge_df(feature_df, action_feature_df)
        feature_df = merge_df(feature_df, order_feature_df)
        feature_df = merge_df(feature_df, action_order_feature_df)

    return feature_df

In [76]:
# get overall labels
def get_label(data, begin_date, end_date):

    # filter by date range
    order_df = data.order_df[(data.order_df.o_date >= begin_date) & (data.order_df.o_date <= end_date)]

    # filter by target cate
    cate_30_101_cond = (order_df.cate == 30) | (order_df.cate == 101)
    order_df = order_df[cate_30_101_cond]

    # transform date label representation and sort
    order_df.o_date = (order_df.o_date - begin_date).apply(lambda x: x.days).astype(int)
    order_df = order_df.sort_values(['user_id', 'o_date'])

    # start building label
    label_df = data.user_df[['user_id']]

    ########## LABEL START ##########

    # order num: 30 101
    label_order_num_df = order_df. \
                        groupby('user_id')['o_id']. \
                        nunique(). \
                        reset_index(). \
                        rename(columns={'o_id': 'order_num'})
    label_df = merge_df(label_df, label_order_num_df)

    # first day of order: 30 101
    label_pred_date_df = order_df. \
                        drop_duplicates('user_id', keep='first') \
                        [['user_id', 'o_date']]. \
                        rename(columns={'o_date': 'pred_date'})
    label_df = merge_df(label_df, label_pred_date_df)

    ########## LABEL END ##########

    label_df = label_df.fillna(0).sort_values(by='user_id')

    return label_df


In [77]:
train_feature = get_feature(data, train_feature_begin_dates, train_feature_end_date, featured_month_periods)
train_label = get_label(data, train_label_begin_date, train_label_end_date)
test_feature = get_feature(data, test_feature_begin_dates, test_feature_end_date, featured_month_periods)